In [1]:
import os
import numpy as np
from cdo import Cdo
import xarray as xr
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as feat
import scipy.constants as co
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import IPython.display as disp
import metpy.interpolate as intp
import pandas as pd
import time
from ipywidgets import IntProgress, HTML

In [2]:
DATADIR = "/scratch2/hugo/ERA5"
CLIMSTOR = "/mnt/climstor/ecmwf/era5/raw"
def filenamescm(y, m, d):
    return [f"{CLIMSTOR}/ML/data/{str(y)}/P{str(y)}{str(m).zfill(2)}{str(d).zfill(2)}_{str(h).zfill(2)}" for h in range(0, 24, 6)]
def filenamecp(y, m, d):
    return [f"{CLIMSTOR}/PL/data/an_pl_ERA5_{str(y)}-{str(m).zfill(2)}-{str(d).zfill(2)}.nc"]  # returns iterable to have same call signature as filenamescl(y, m, d)
def fncm(date):  # instead takes pandas.timestamp as input
    return filenamescm(date.year, date.month, date.day)
def fncp(date):
    return filenamecp(date.year, date.month, date.day)
a = 6.371e6  # m
OMEGA = 7.2921e-5  # rad.s-1

def degcos(x):
    return np.cos(x / 180 * np.pi)
def degsin(x):
    return np.sin(x / 180 * np.pi)
KAPPA = co.R / 1.0035
DATERANGEPL = pd.date_range("19590101", "20211231")
YEARSPL = np.unique(DATERANGEPL.year)
DATERANGEML = pd.date_range("19770101", "20211231")
cdo = Cdo(debug=1)

In [3]:
longname = {
    "u": "U-component of wind",
    "v": "V-component of wind",
    "w": "W-component of wind",
    "z": "Geopotential",
    "t": "Temperature",
    "vo": "Relative vorticity",
    "q": "Specific humidity",
    "r": "relative humidity"
    
}
variablemap = {
    f"{var}{lev}": [var, "PL", lev, f"{longname[var]} at {lev} hPa"] 
    for var in ["u", "v", "vo"] 
    for lev in range(700, 901, 50)
}
var = "z"
vm2 = {
    f"{var}{lev}": [var, "PL", lev, f"{longname[var]} at {lev} hPa"] 
    for lev in [300, 500]
}

variablemap.update(vm2)
variablemap["t850"] = ["t", "PL", 850, f"{longname['t']} at 850 hPa"]

In [ ]:
# create yearly files containing u(700-75-925), compliant with Barripedro et. al (2022); Woollings (2010)

f1 = IntProgress(min=0, max=len(YEARSPL) * 12)
display(f1)
for year in YEARSPL:
    for month in range(1, 13):
        fn = f"{DATADIR}/Woollings/{year}{str(month).zfill(2)}.nc"
        if os.path.isfile(fn):
            f1.value += 1
            continue
        to_merge = []
        for d in DATERANGEPL[np.logical_and(DATERANGEPL.year == year, DATERANGEPL.month == month)]:    
            to_merge.extend(fncp(d))
        xr.open_mfdataset(
            to_merge, drop_variables=[v for v in longname if v != "u"]
            ).sel(
                level=[700.0, 750.0, 800.0, 850.0, 900.0]
                ).to_netcdf(fn)
        f1.value += 1

In [4]:
# dynamical tropopause calculations, to be scaled on piz daint probably
ds = xr.open_dataset(filenamescm(1979, 1, 1)[0], engine="netcdf4")

In [12]:
ds["vort"] = (ds["V"].differentiate("lon") - (ds["U"] * degcos(ds["lat"])).differentiate("lat")) / a / degcos(ds["lat"])
ds["absvort"] = ds["vort"] + 2 * OMEGA * degsin(ds["lat"])

In [6]:
P = np.tensordot(ds["PS"][:, 0, :, 0], ds["hybm"], axes=0) 

In [ ]:
def create_plot(to_plot, titles, levels, twolevel=False, startindex=-1):
    # Figure
    transform = ccrs.PlateCarree()
    projection = transform
    if twolevel:
        fig, axes = plt.subplots(
            2,
            int(len(to_plot) / 2),
            subplot_kw={"projection": projection}, constrained_layout=True #, figsize=(6 * len(to_plot) // 2, 13)
        )
    else:
        fig, axes = plt.subplots(
            1, len(to_plot), subplot_kw={"projection": projection}, constrained_layout=True
        )
    axes = np.atleast_1d(axes)
    axes = axes.flatten()

    # Add coastline and boarders
    coastline = feat.NaturalEarthFeature(
        "physical", "coastline", "10m", edgecolor="black", facecolor="none"
    )
    borders = feat.NaturalEarthFeature(
        "cultural",
        "admin_0_boundary_lines_land",
        "10m",
        edgecolor="grey",
        facecolor="none",
    )
    plt_rej = []
    cbar = [None] * len(to_plot)
    for j in range(len(to_plot)):
        ax = axes[j]
        plt_rej.append(
            ax.contourf(
                to_plot[j]["lon"],
                to_plot[j]["lat"],
                to_plot[j].isel(time=startindex),
                levels=levels[j],
                transform=transform,
                transform_first=True,
                # cmap=cmap,
                zorder=0,
            )
        )

        ax.add_feature(coastline)
        ax.add_feature(borders)
        ax.set_xmargin(0)
        ax.set_ymargin(0)
        ax.set_title(f"Day {startindex}, {titles[j]}, g.a : {np.mean(to_plot[j][startindex]):.2f}")

        cbar[j] = fig.colorbar(plt_rej[j], ax=ax,fraction=0.046, pad=0.04)

    def animate_all(i):
        global plt_rej
        for j in range(len(to_plot)):
            ax = axes[j]
            for c in plt_rej[j].collections:
                c.remove()
            plt_rej[j] = ax.contourf(
                to_plot[j]["lon"],
                to_plot[j]["lat"],
                to_plot[j].isel(time=i),
                levels=levels[j],
                transform=transform,
                transform_first=True,
                # cmap=cmap,
                zorder=0,
            )
            ax.set_title(f"Day {i + 1}, {titles[j]}, g.a : {np.mean(to_plot[j][i]):.2f}")
            cbar[j] = fig.colorbar(plt_rej[j], cax=fig.axes[len(axes) + j])
        return plt_rej

    return fig, axes, plt_rej, animate_all